# <span style="color:green"> Import libraries<span style="color:green">

In [1]:
#Import all the required libraries
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn 
import random
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pybiomart
from biomart import BiomartServer
from sklearn.feature_selection import RFECV
import xgboost as xgb

# <span style="color:green"> ML for Feature Selection and Association <span style="color:green">

In [2]:
#Read the file 
df= pd.read_table("/home/Vaishnavi/Imputeted_data/Imputed_maf_0.01/GWAStutorialldlraw.raw", delim_whitespace= True)
print(df.head())
print("Shape of file", df.shape)

     FID  IID  PAT  MAT  SEX  PHENOTYPE  rs2436029:269101:C:T_T  \
0  10004    1    0    0    2         75                       2   
1  10005    1    0    0    1         69                       2   
2  10007    1    0    0    1        108                       2   
3  10008    1    0    0    1         94                       2   
4  10009    1    0    0    1         92                       2   

   rs117942522:320955:C:A_A  rs17158845_T  rs4881331:517451:C:G_G  ...  \
0                         0             2                       2  ...   
1                         0             2                       2  ...   
2                         0             2                       2  ...   
3                         0             2                       2  ...   
4                         0             2                       2  ...   

   rs4880117_A  rs11137286_T  rs11137126_G  rs9410127_C  rs3125786_T  \
0            2             2             2            2            2   
1       

In [3]:
new_col_names= list(df.columns[:6])
new_col_names.extend([col_name[:-2] for col_name in df.columns[6:]])
df.columns=new_col_names
df.columns

Index(['FID', 'IID', 'PAT', 'MAT', 'SEX', 'PHENOTYPE', 'rs2436029:269101:C:T',
       'rs117942522:320955:C:A', 'rs17158845', 'rs4881331:517451:C:G',
       ...
       'rs4880117', 'rs11137286', 'rs11137126', 'rs9410127', 'rs3125786',
       'rs11137198', 'rs7865267', 'rs1378954:141005656:G:A', 'rs9777369',
       '9:141049179:C:T'],
      dtype='object', length=38836)

In [4]:
#X is Independent feature and y is dependent feature
X= df.iloc[:, 6:]
print("Shape of X", X.shape)
y= df.PHENOTYPE
print("Shape of y", y.shape)

Shape of X (804, 38830)
Shape of y (804,)


## <span style="color:green"> LASSO <span style="color:green">

In [5]:
# Create and fit the LASSOCV model select 10,000 SNP
lasso_cv = linear_model.LassoCV(alphas= [0.000065], cv=5, random_state=2000)

# Fit the model to the data
lasso_cv.fit(X,y)

#Print the optimal alpha value
print ("Optimal alpha:", lasso_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
lasso_snps_ldl = np.where(lasso_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(lasso_snps_ldl), "selected SNPs")

#Final SNPs
l1= X.iloc[:,lasso_snps_ldl]
print("print shape of selected snps by lasso",l1.shape)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.538e+02, tolerance: 8.481e+01
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.497e+02, tolerance: 8.188e+01
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 6.5e-05
5055 selected SNPs
print shape of selected snps by lasso (804, 5055)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.909e+02, tolerance: 9.977e+01
  model = cd_fast.enet_coordinate_descent(


### Linear regression

In [6]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(l1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

f_statistic, p_value = f_regression(l1, Y)
selected_p_indices = np.where(p_value < 5e-04)[0]
selected_p_values = p_value[selected_p_indices]
print("Selected p-values:")
print(selected_p_values)
print("Number of selected features:", len(selected_p_indices))
selected_f_values = f_statistic[selected_p_indices]
print("Selected F-values:")
print(selected_f_values)
top_f_indices = np.argsort(-selected_f_values)
print("Indices of selected features with top F-values:")
print(top_f_indices)
top_f_features = l1.iloc[:, selected_p_indices[top_f_indices]]
print("Top selected features:")
print(top_f_features)
lasso_LR_sigSNP=l1.iloc[:, selected_p_indices]
lasso_LR_sigSNP
lasso_LR_sig_col= list(lasso_LR_sigSNP.columns)
lasso_LR_sig_col

Y.shape (804,)
x_train=(562, 5055), x_test=(242, 5055), Y_train= (562,), Y_test= (242,)
r2 is  -7.062706566016006
mean_absolute_error 24.622159361950125
mean_squared_error  1226.6967750084948
explained_variance_score 0.21415028471685826
Selected p-values:
[1.01519932e-04 2.96074422e-04 3.61043914e-05 3.94177532e-04
 1.85956925e-04 2.26316511e-04 2.23764587e-04 4.61215627e-04
 3.92748510e-05 5.56941232e-05]
Number of selected features: 10
Selected F-values:
[15.26101612 13.21084912 17.25964377 12.66639828 14.09911231 13.72347453
 13.74514156 12.36832641 17.09631287 16.41964858]
Indices of selected features with top F-values:
[2 8 9 0 4 6 5 1 3 7]
Top selected features:
     rs1778332  rs9844297  rs114142749:60004623:C:T  rs17363282  rs1270640  \
0            2          2                         0           2          2   
1            2          2                         0           2          2   
2            2          2                         0           2          2   
3          

['rs17363282',
 'rs17148508',
 'rs1778332',
 'rs1337321',
 'rs1270640',
 'rs17368143',
 'rs17046473',
 'rs7580907',
 'rs9844297',
 'rs114142749:60004623:C:T']

### Random Forest

In [7]:
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)


# Fit RFECV on the training data
rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))


r2 is  -123.1850653760677
mean_absolute_error 28.389087581374696
mean_squared_error  1527.7362754472292


### Support Vector Machine

In [8]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))



r2 is  -3.460697449509161
mean_absolute_error 24.248212205770606
mean_squared_error  1172.5364483292863
explained_variance_score 0.2517321677242027


### XGBoost

In [9]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -12.14543834478865
mean_absolute_error 28.69527681208839
mean_squared_error  1570.4655811551386
explained_variance_score -0.007063029289272826


## <span style="color:green"> Ridge<span style="color:green">

In [10]:
# Create and fit the RidgeCV model select 5000 SNP
ridge_cv = linear_model.RidgeCV(alphas=[0.005], cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X, y)

# Optimal alpha value
print("Optimal alpha:", ridge_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
ridge_snps_ldl = np.where(ridge_cv.coef_)[0]

# Select coefficients with non-zero values
selected_features = ridge_cv.coef_ != 0

# Select coefficients of selected SNPs
coefficients = ridge_cv.coef_[selected_features]

# Get the indices of the top 5000 SNPs based on descending coefficients
top_n = 5000
sorted_indices = np.argsort(-coefficients)
selected_indices = sorted_indices[:top_n]

# Print the indices of selected SNPs and the number of selected SNPs
print("Selected SNPs indices based on top coefficients:", selected_indices)
print(len(selected_indices), "selected SNPs")

#Maximum Coefficient
print("Maximum value of coefficient", coefficients.max())

#Final SNPs
R1= X.iloc[:,selected_indices]
print("print shape of selected snps by Ridge",R1.shape)


Optimal alpha: 0.005
Selected SNPs indices based on top coefficients: [24344 11183 24683 ... 30323 11320 14165]
5000 selected SNPs
Maximum value of coefficient 0.43818829476400195
print shape of selected snps by Ridge (804, 5000)


### Linear Regression

In [11]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(R1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (804,)
x_train=(562, 5000), x_test=(242, 5000), Y_train= (562,), Y_test= (242,)


In [12]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

f_statistic, p_value = f_regression(R1, Y)
selected_p_indices = np.where(p_value < 5e-04)[0]
selected_p_values = p_value[selected_p_indices]
print("Selected p-values:")
print(selected_p_values)
print("Number of selected features:", len(selected_p_indices))
selected_f_values = f_statistic[selected_p_indices]
print("Selected F-values:")
print(selected_f_values)
top_f_indices = np.argsort(-selected_f_values)
print("Indices of selected features with top F-values:")
print(top_f_indices)
top_f_features = R1.iloc[:, selected_p_indices[top_f_indices]]
print("Top selected features:")
print(top_f_features)
Ridge_LR_sigSNP=R1.iloc[:, selected_p_indices]
Ridge_LR_sigSNP
Ridge_LR_sig_col= list(Ridge_LR_sigSNP.columns)
Ridge_LR_sig_col

r2 is  0.23516675371052276
mean_absolute_error 14.901300123141873
mean_squared_error  426.0674752708918
explained_variance_score 0.7267570487324846
Selected p-values:
[0.00011675 0.00030055 0.0003986  0.00025274 0.00034545 0.0004939
 0.00011867]
Number of selected features: 7
Selected F-values:
[14.99217683 13.18225393 12.64520192 13.5125931  12.91722134 12.23854868
 14.96076177]
Indices of selected features with top F-values:
[0 6 3 1 4 2 5]
Top selected features:
     rs114960359:117239408:C:A  rs10056040  rs429717:46317839:A:G  rs7250848  \
0                            0           2                      2          1   
1                            0           2                      1          2   
2                            0           2                      2          2   
3                            0           2                      1          2   
4                            0           2                      2          2   
..                         ...         ...        

['rs114960359:117239408:C:A',
 'rs7250848',
 'rs10046533:75286795:C:G',
 'rs429717:46317839:A:G',
 '4:142689553:G:A',
 'rs17140133',
 'rs10056040']

### Random Forest

In [13]:
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)


# Fit RFECV on the training data
rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))

r2 is  -69.57598214428167
mean_absolute_error 29.072424524803633
mean_squared_error  1550.744807350736


### Support Vector Machine

In [14]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  0.45221030716232424
mean_absolute_error 13.916722121066252
mean_squared_error  377.0827072063047
explained_variance_score 0.7613266921803723


### XGBoost

In [15]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -14.008014965332777
mean_absolute_error 30.172190737133185
mean_squared_error  1705.5436353245773
explained_variance_score -0.09289767366345614


## <span style="color:green"> Elastic net<span style="color:green">

In [16]:
# Create and fit the Elasticnet model
elastic_cv = linear_model.ElasticNetCV(l1_ratio=[0.5],alphas=[0.136], cv=5, random_state= 2000)
elastic_cv.fit(X, y)

# Print the optimal alpha value
print ("Optimal alpha:", elastic_cv.alpha_)
print("Optimal l1_ratio: ", elastic_cv.l1_ratio_)

# Indices of the SNPs with non-zero coefficients
elastic_snps_ldl = np.where(elastic_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(elastic_snps_ldl), "selected SNPs")

#Final SNPs
E1= X.iloc[:,elastic_snps_ldl]
print("print shape of selected snps by Elastic net",E1.shape)

Optimal alpha: 0.136
Optimal l1_ratio:  0.5
5000 selected SNPs
print shape of selected snps by Elastic net (804, 5000)


### Linear Regression

In [17]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(E1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (804,)
x_train=(562, 5000), x_test=(242, 5000), Y_train= (562,), Y_test= (242,)


In [18]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

f_statistic, p_value = f_regression(E1, Y)
selected_p_indices = np.where(p_value < 5e-04)[0]
selected_p_values = p_value[selected_p_indices]
print("Selected p-values:")
print(selected_p_values)
print("Number of selected features:", len(selected_p_indices))
selected_f_values = f_statistic[selected_p_indices]
print("Selected F-values:")
print(selected_f_values)
top_f_indices = np.argsort(-selected_f_values)
print("Indices of selected features with top F-values:")
print(top_f_indices)
top_f_features = E1.iloc[:, selected_p_indices[top_f_indices]]
print("Top selected features:")
print(top_f_features)
Elasticnet_LR_sigSNP=E1.iloc[:, selected_p_indices]
Elasticnet_LR_sigSNP
Elasticnet_LR_sig_col= list(Elasticnet_LR_sigSNP.columns)
Elasticnet_LR_sig_col

r2 is  0.5884972895966444
mean_absolute_error 10.754641248473234
mean_squared_error  256.45290776741444
explained_variance_score 0.8355334697053634
Selected p-values:
[1.01519932e-04 2.96074422e-04 3.61043914e-05 3.94177532e-04
 1.85956925e-04 2.52742690e-04 1.60126498e-04 2.86448156e-04
 3.77362185e-04 2.32562385e-04 3.00551798e-04 3.49989000e-05
 1.67826003e-04 2.20980705e-04 3.01483324e-04 2.26316511e-04
 1.34327091e-04 5.17290623e-05 2.17861807e-04 2.23764587e-04
 2.03221057e-04 4.61215627e-04 4.20672054e-05 1.68016657e-04
 9.95898777e-05 3.92748510e-05 5.56941232e-05 1.16746301e-04
 3.45449460e-04 1.36877060e-04 1.18669783e-04 5.69937705e-05
 8.08387343e-05 7.92374209e-07 4.93586322e-04 1.34742036e-04
 1.50831357e-04 4.93902383e-04 7.94332590e-06 2.72921870e-04
 3.88784056e-04 3.98601170e-04 3.34552347e-04 4.80749141e-04
 1.68597540e-04 7.90462621e-05 4.77655973e-04]
Number of selected features: 47
Selected F-values:
[15.26101612 13.21084912 17.25964377 12.66639828 14.09911231 13.

['rs17363282',
 'rs17148508',
 'rs1778332',
 'rs1337321',
 'rs1270640',
 'rs429717:46317839:A:G',
 'rs4390610',
 'rs2111661',
 'rs9961679',
 'rs16979982',
 'rs7250848',
 'rs12565884',
 'rs13433317',
 'rs845984',
 'rs9611598',
 'rs17368143',
 'rs17466952',
 'rs6721324',
 'rs13431528',
 'rs17046473',
 'rs7573764',
 'rs7580907',
 'rs6431555',
 'rs11707474:26174707:A:T',
 'rs11713930',
 'rs9844297',
 'rs114142749:60004623:C:T',
 'rs114960359:117239408:C:A',
 '4:142689553:G:A',
 'rs10045328',
 'rs10056040',
 'rs10476900',
 'rs4840245',
 'rs17061291',
 'rs7454060',
 'rs9968989',
 'rs9498157:149102500:A:T',
 'rs17140133',
 'rs12673401',
 'rs936857',
 'rs11977502',
 'rs10046533:75286795:C:G',
 'rs17156252',
 'rs3757557',
 'rs17170216',
 'rs7829831',
 'rs4236776']

### Random Forest

In [19]:
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)


# Fit RFECV on the training data
rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  -83.35238199831839
mean_absolute_error 28.264004281554012
mean_squared_error  1462.696816865877
explained_variance_score 0.06239113068281965


### Support Vector Regression

In [20]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))
perm_importance= permutation_importance(svr, x_test, Y_test, n_repeats=10, random_state=2000, n_jobs=70)


imp_score=perm_importance.importances_mean
feature_indices = np.argsort(np.abs(imp_score))[::-1]
print(f"Selected feature indices are {feature_indices}")
# # Specify the number of features you want to select based on magnitude
num_selected_features = 100
top_100=feature_indices[:num_selected_features]
print(f"Top 100 fearure indices are {feature_indices}")
imp_score_100=imp_score[top_100]
print(f"Top 100 fearures permutation scores are {imp_score_100}")
Elasticnet_SVR_sigSNP= E1.iloc[:,top_100]
Elasticnet_SVR_sig_col= list(Elasticnet_SVR_sigSNP.columns)
Elasticnet_SVR_sig_col

r2 is  0.788886164228993
mean_absolute_error 8.658024483347209
mean_squared_error  171.8369404477901
explained_variance_score 0.8897983479276816
Selected feature indices are [4048 2073 2436 ... 1984 1175 3505]
Top 100 fearure indices are [4048 2073 2436 ... 1984 1175 3505]
Top 100 fearures permutation scores are [ 0.00130108  0.00120506  0.00117774  0.00117456  0.0011215   0.00108006
  0.00107939  0.00106489  0.00083367  0.00082445  0.00081813  0.00079621
  0.0007887   0.00077276  0.0007641   0.00075778  0.00075758  0.00074712
  0.00073069  0.00072727  0.00071748  0.00071204  0.0007059   0.00069548
  0.00069486  0.0006945   0.00067777  0.00067501  0.00067143  0.00066202
  0.00065973  0.00065951  0.00065563  0.00064964  0.00064494  0.00063843
  0.00063713  0.00063529  0.00063403  0.00063223  0.00062565  0.0006178
  0.00061678  0.00061621  0.00061439  0.00060744  0.00060443 -0.00059452
  0.00059258 -0.00058944  0.00058609  0.00058195  0.00057295  0.00057166
  0.00056874 -0.00056767  0.00

['rs4840245',
 'rs12565884',
 'rs9611598',
 'rs11707474:26174707:A:T',
 'rs4390610',
 'rs9498157:149102500:A:T',
 'rs11683778',
 'rs6721324',
 'rs16871601',
 'rs16956998:82325795:A:T',
 'rs4236776',
 'rs936857',
 'rs17046473',
 'rs7250848',
 'rs7007853',
 'rs11114094',
 'rs2111661',
 'rs17466952',
 'rs12576190:80938399:G:C',
 'rs1778332',
 'rs328016',
 'rs7573764',
 'rs17148508',
 'rs17237145',
 'rs12673401',
 'rs10056040',
 'rs4538947',
 'rs75404656:181460531:G:A',
 'rs968846',
 'rs10046533:75286795:C:G',
 'rs10045328',
 'rs12595894:80105008:C:T',
 'rs7454060',
 'rs1412922',
 'rs6502978',
 'rs11863312',
 'rs8102985',
 'rs4987765',
 'rs114142749:60004623:C:T',
 'rs1532468:109684816:T:A',
 'rs8006542',
 'rs9844297',
 'rs3757557',
 'rs9479837',
 'rs7829831',
 'rs7580907',
 'rs16845462',
 'rs7404477',
 'rs8192049',
 'rs16837404',
 'rs1516242',
 'rs6485709',
 'rs56390351:184996010:C:T',
 'rs2301719',
 'rs7599185:204583597:T:C',
 'rs8055769',
 'rs16882715',
 'rs17368143',
 'rs242764',
 'rs8

### XGBoost for Elastic-net

In [21]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -16.674891882959155
mean_absolute_error 28.056912634983536
mean_squared_error  1469.7270667198777
explained_variance_score 0.05781458120083338


## <span style="color:green"> Mutual information <span style="color:green">

In [22]:
# Create and fit the Mutual_info model
mutual_info= mutual_info_regression(X, y, random_state=2000)
print("Length of mutal info", len(mutual_info))
mutual_info= pd.Series(mutual_info, index=df.columns[6:])

#Sort mutual_info values descending order
vdesc= mutual_info.sort_values(ascending=False)
vdesc_nonzero= vdesc[vdesc!=0]
print("Maximum value of mutualinfo", vdesc_nonzero.max)
print("Length of mutal info non zero", len(vdesc_nonzero))
k= np.where(vdesc_nonzero)
first_col= k[0]

#Selected 5000 SNPs
top5000 = 5000
mi_pos= first_col[:top5000]

#Final SNPs
M1= X.iloc[:,mi_pos]
print("print shape of selected snps by Mutual_info",M1.shape)

Length of mutal info 38830
Maximum value of mutualinfo <bound method NDFrame._add_numeric_operations.<locals>.max of rs9881090                  9.749381e-02
rs12649176                 9.574809e-02
rs16824778                 9.354285e-02
rs17681524                 9.232919e-02
rs1472491                  9.055629e-02
                               ...     
rs17029090                 2.040916e-06
rs56002751:39625366:A:C    8.888933e-07
rs12555893                 5.370780e-07
rs966078                   5.363469e-07
rs7523161                  2.856900e-07
Length: 19050, dtype: float64>
Length of mutal info non zero 19050
print shape of selected snps by Mutual_info (804, 5000)


### Linear Regression

In [23]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(M1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (804,)
x_train=(562, 5000), x_test=(242, 5000), Y_train= (562,), Y_test= (242,)


In [24]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

f_statistic, p_value = f_regression(M1, Y)
selected_p_indices = np.where(p_value < 5e-04)[0]
selected_p_values = p_value[selected_p_indices]
print("Selected p-values:")
print(selected_p_values)
print("Number of selected features:", len(selected_p_indices))
selected_f_values = f_statistic[selected_p_indices]
print("Selected F-values:")
print(selected_f_values)
top_f_indices = np.argsort(-selected_f_values)
print("Indices of selected features with top F-values:")
print(top_f_indices)
top_f_features = M1.iloc[:, selected_p_indices[top_f_indices]]
print("Top selected features:")
print(top_f_features)
MI_LR_sigSNP=M1.iloc[:, selected_p_indices]
MI_LR_sigSNP
MI_LR_sig_col= list(MI_LR_sigSNP.columns)
MI_LR_sig_col

r2 is  -8.967661422136974
mean_absolute_error 29.355031468962512
mean_squared_error  1659.9338019869726
explained_variance_score -0.06444087675509103
Selected p-values:
[1.01519932e-04 2.96074422e-04 3.61043914e-05 3.94177532e-04
 1.85956925e-04]
Number of selected features: 5
Selected F-values:
[15.26101612 13.21084912 17.25964377 12.66639828 14.09911231]
Indices of selected features with top F-values:
[2 0 4 1 3]
Top selected features:
     rs1778332  rs17363282  rs1270640  rs17148508  rs1337321
0            2           2          2           2          2
1            2           2          2           2          2
2            2           2          2           2          2
3            2           2          2           2          2
4            2           2          2           2          2
..         ...         ...        ...         ...        ...
799          2           2          2           2          2
800          2           1          2           2          2
801      

['rs17363282', 'rs17148508', 'rs1778332', 'rs1337321', 'rs1270640']

### Random Forest

In [25]:
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  -106.02197376783775
mean_absolute_error 29.068933183934003
mean_squared_error  1559.4023373704358
explained_variance_score -3.932338212253583e-05


### Support Vector Regressor

In [26]:
svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale')
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 is ", score)
h= mean_absolute_error(y_pred,Y_test)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(y_pred, Y_test))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  -5.111672379202908
mean_absolute_error 30.620929464937387
mean_squared_error  1758.2542849613203
explained_variance_score -0.1259701588287958


### XGBoost 

In [27]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(y_pred, Y_test)
print("r2 score",score)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -12.253183018852267
mean_absolute_error 30.78732646595348
mean_squared_error  1683.8101853904764
explained_variance_score -0.07982777063303526
